In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [2]:
hps = utils.get_hparams_from_file("./configs/ms_ms.json")

In [3]:
net_g = SynthesizerTrn(
    
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("./logs/ms_ms/G_18000.pth", net_g, None)
# _ = utils.load_checkpoint("logs/ljs_mb_istft_vits/G_1000000.pth", net_g, None)

Mutli-stream iSTFT VITS


In [9]:
import time
def tts(txt,device="cuda"):
    
    stn_tst = get_text(txt, hps)
    with torch.no_grad():
        x_tst = stn_tst.unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
        spk = torch.LongTensor([3])
        t1 = time.time()
        audio = net_g.to(device).infer(x_tst.to(device), x_tst_lengths.to(device),sid=spk, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].cpu().data.float().numpy()
        t2 = time.time()
        print("推理时间：", (t2-t1),"s")
    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
# tts("[ZH]大家好啊，我是说的道理，今天来点大家想看的东西[ZH]",device="cpu")


In [6]:
# tts("[JA]なんでこんなに慣れてんのよ。私の方が先に好きだったのに[JA]",device="cuda")
# tts("[JA]こんにちは。私わあやちねねです。こんにちは。私わあやちねねです。こんにちは。私わあやちねねです。[JA]",device="cuda")


In [10]:
# tts("[JA]なんでこんなに慣れてんのよ。私の方が先に好きだったのに[JA]",device="cpu")
# tts("[JA]こんにちは。私わあやちねねです。こんにちは。私わあやちねねです。こんにちは。私わあやちねねです。[JA]",device="cpu")


In [11]:
# tts("[JA]そうだ。俺たちが今まで积み上げてきたもんは全部无駄じゃなかった。こからも俺たちが立ち止まらないかぎり道は続く[JA]")
